<a href="https://colab.research.google.com/github/onthegoshahbaaz/digit_recognization/blob/main/digitrecocnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import struct
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
# Function to read IDX files
def read_idx(file_path):
    with open(file_path, 'rb') as f:
        magic, num_items = struct.unpack(">II", f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
    return labels

def read_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
    return images

In [4]:
# Paths to the dataset files
train_labels_path = '/content/train-labels-idx1-ubyte'
test_labels_path = '/content/t10k-labels-idx1-ubyte'
train_images_path = '/content/train-images-idx3-ubyte'
test_images_path = '/content/t10k-images-idx3-ubyte'

In [5]:
# Load datasets
train_labels = read_idx(train_labels_path)
test_labels = read_idx(test_labels_path)
train_images = read_images(train_images_path)
test_images = read_images(test_images_path)

In [6]:
# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

In [7]:
# Reshape the images for the CNN
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

In [8]:
# Convert labels to categorical format
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
844/844 [==============================] - 46s 53ms/step - loss: 0.1889 - accuracy: 0.9433 - val_loss: 0.0527 - val_accuracy: 0.9853
Epoch 2/5
844/844 [==============================] - 43s 51ms/step - loss: 0.0545 - accuracy: 0.9830 - val_loss: 0.0453 - val_accuracy: 0.9880
Epoch 3/5
844/844 [==============================] - 44s 52ms/step - loss: 0.0389 - accuracy: 0.9876 - val_loss: 0.0408 - val_accuracy: 0.9885
Epoch 4/5
844/844 [==============================] - 46s 55ms/step - loss: 0.0299 - accuracy: 0.9908 - val_loss: 0.0389 - val_accuracy: 0.9897
Epoch 5/5
844/844 [==============================] - 46s 55ms/step - loss: 0.0239 - accuracy: 0.9926 - val_loss: 0.0349 - val_accuracy: 0.9897


In [12]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 4s 12ms/step - loss: 0.0306 - accuracy: 0.9899
Test accuracy: 0.9898999929428101


In [13]:
# Predict the labels for test images
predicted_labels = model.predict(test_images)
predicted_labels = np.argmax(predicted_labels, axis=1)
true_labels = np.argmax(test_labels, axis=1)

313/313 [==============================] - 2s 8ms/step


In [14]:
# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[ 976    0    1    0    0    0    2    1    0    0]
 [   0 1130    2    1    0    0    1    1    0    0]
 [   1    0 1023    1    1    0    0    6    0    0]
 [   0    0    0 1004    0    3    0    2    1    0]
 [   0    0    0    0  979    0    2    0    0    1]
 [   1    0    1    5    0  881    1    1    1    1]
 [   2    2    0    0    1    3  950    0    0    0]
 [   0    1    5    0    0    1    0 1021    0    0]
 [   6    0    6    2    2    2    0    7  944    5]
 [   0    2    0    0    5    5    1    5    0  991]]


In [15]:
# Compute accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9899
